1.  **Approche Statistique (SARIMAX)** : Comprendre les dynamiques temporelles et l'impact des fondamentaux (Charge, Vent, Solaire).
2.  **Approche Machine Learning (LightGBM)** : Capturer les non-linéarités complexes du marché.
3.  **Analyse de la Volatilité** : Expliquer les pics de prix par la théorie économique du "Merit Order".

---

In [10]:
import os
import joblib
from datetime import datetime
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings

warnings.filterwarnings('ignore')

## Chargement de données features 

In [2]:
df = pd.read_csv('../../data/processed/df_features_france_2015_2017.csv', parse_dates=['utc_timestamp'], index_col='utc_timestamp')
target_col = 'price_day_ahead'
print(f"Dataset chargé : {df.shape} lignes.")
df.head()

Dataset chargé : (25560, 37) lignes.


,load,load_forecast,solar,wind,price_day_ahead,temperature,cloud_cover,nuclear,wind_speed,hour,...,price_rolling_std_6h,load_rolling_mean_6h,price_rolling_mean_24h,price_rolling_std_24h,load_rolling_mean_24h,renewable_generation,total_generation,price_delta,load_x_hour,temp_x_cloud
utc_timestamp,,,,,,,,,,,,,,,,,,,,,
2015-01-06 00:00:00+00:00,67795.0,67900.0,0.0,739.0,45.21,3.257507,0.395513,58930.0,2.012984,0,...,4.663818,71753.333333,52.661667,10.753981,71452.291667,739.0,59669.0,-4.57,0.00,1.288386
2015-01-06 01:00:00+00:00,66811.0,67450.0,0.0,736.0,38.80,3.114288,0.422361,58885.0,1.987322,1,...,3.420566,70178.166667,52.916250,10.488004,71726.958333,736.0,59621.0,-6.41,668.11,1.315353
2015-01-06 02:00:00+00:00,64040.0,64850.0,0.0,753.0,35.00,2.995453,0.413952,58497.0,1.989497,2,...,4.798436,68875.166667,53.065833,10.247332,71998.000000,753.0,59250.0,-3.80,1280.80,1.239975
2015-01-06 03:00:00+00:00,63275.0,63650.0,0.0,754.0,33.43,2.959351,0.433965,57890.0,1.973881,3,...,6.751242,67421.500000,53.172917,10.033511,72264.000000,754.0,58644.0,-1.57,1898.25,1.284255
2015-01-06 04:00:00+00:00,65977.0,65300.0,0.0,823.0,37.58,2.842896,0.459675,58227.0,2.030342,4,...,7.540150,66305.500000,53.184583,10.009351,72520.416667,823.0,59050.0,4.15,2639.08,1.306807


## Modélisation Statistique : SARIMAX

### Pourquoi SARIMAX ?
Le modèle **SARIMA** (Seasonal AutoRegressive Integrated Moving Average) permet de modéliser une série temporelle en se basant sur son passé (AutoRegressive) et ses erreurs passées (Moving Average), tout en gérant la saisonnalité (Seasonal).

Cependant, le prix de l'électricité n'est pas juste une suite de nombres abstraits : il est piloté par des réalités physiques. C'est pourquoi nous utilisons **SARIMAX** (avec **X** pour eXogenous variables).

### Les Variables Explicatives (Exogènes)
Nous intégrons ici les facteurs fondamentaux du marché :
-   **La Consommation (`load`)** : C'est le facteur #1. Plus la demande est forte, plus on doit activer des centrales coûteuses (gaz/charbon).
-   **La Production Renouvelable & Météo** : `wind` et `solar` sont les productions, mais `wind_speed` et `cloud_cover` (nébulosité) sont les causes racines météorologiques. Les ajouter augmente la précision.
-   **Le Nucléaire (`nuclear`)** : Base de la production française, offre stable et peu coûteuse.
-   **La Température (`temperature`)** : Driver indirect de la demande (chauffage/clim).


In [3]:
# Sélection des variables exogènes disponibles
exog_vars = ['load', 'solar', 'wind', 'nuclear', 'temperature', 'cloud_cover', 'wind_speed']
available_exog = [c for c in exog_vars if c in df.columns]
print(f"Variables Exogènes utilisées : {available_exog}")

# Agrégation journalière (Moyenne)
cols_to_use = [target_col] + available_exog
df_daily = df[cols_to_use].resample('D').mean().dropna()

# Split Chronologique (On garde la fin pour le test)
train_size = int(len(df_daily) * 0.95)
train, test = df_daily.iloc[:train_size], df_daily.iloc[train_size:]

y_train = train[target_col]
X_train = train[available_exog]
y_test = test[target_col]
X_test = test[available_exog]

print(f"Entraînement sur {len(train)} jours. Test sur {len(test)} jours.")

# Modèle SARIMAX (1,1,1) x (0,1,1,7)
# Saisonnalité hebdomadaire (7) car on observe souvent des cycles semaine/week-end
model_sarima = SARIMAX(y_train, exog=X_train, order=(1, 1, 1), seasonal_order=(0, 1, 1, 7))
results_sarima = model_sarima.fit(disp=False)


Variables Exogènes utilisées : ['load', 'solar', 'wind', 'nuclear', 'temperature', 'cloud_cover', 'wind_speed']
Entraînement sur 1011 jours. Test sur 54 jours.


In [4]:

# Prédiction
forecast = results_sarima.get_forecast(steps=len(test), exog=X_test)
mean_forecast = forecast.predicted_mean
conf_int = forecast.conf_int()

# Scores
mae = mean_absolute_error(y_test, mean_forecast)
rmse = root_mean_squared_error(y_test, mean_forecast)
r2 = r2_score(y_test, mean_forecast)
mape = mean_absolute_percentage_error(y_test, mean_forecast)
print(f"MAE: {mae:.2f}€/MWh | RMSE: {rmse:.2f} | R2: {r2:.2f} | MAPE: {mape:.2f}")


MAE: 4.76€/MWh | RMSE: 6.93 | R2: 0.65 | MAPE: 0.07


In [5]:
# Visualisation
fig = go.Figure()
train_visu = y_train.iloc[-180:] # Zoom sur les derniers mois du train
fig.add_trace(go.Scatter(x=train_visu.index, y=train_visu, name='Historique (Train)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=y_test.index, y=y_test, name='Réalité (Test)', line=dict(color='green')))
fig.add_trace(go.Scatter(x=y_test.index, y=mean_forecast, name='Prédiction SARIMAX', line=dict(color='red', dash='dash')))
fig.add_trace(go.Scatter(
    x=y_test.index.tolist() + y_test.index.tolist()[::-1],
    y=conf_int.iloc[:, 1].tolist() + conf_int.iloc[:, 0].tolist()[::-1],
    fill='toself', fillcolor='rgba(255, 0, 0, 0.2)', line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip", showlegend=True, name='Intervalle de Confiance 95%'
))
fig.update_layout(title=f'SARIMAX (MAE: {mae:.2f} €)', template='plotly_white')
fig.show()

## Modélisation Avancée : LightGBM

Contrairement à SARIMAX qui est un modèle linéaire (une ligne droite + des cycles), LightGBM est un modèle à base d'arbres de décision (Gradient Boosting). Voici pourquoi c'est crucial pour l'électricité :

1.  **Gestion des Effets de Seuil (Non-linéarité)** :
    *   *Logique linéaire (SARIMAX)* : "Si la demande augmente de 1GW, le prix augmente de 10€, quel que soit le niveau actuel."
    *   *Réalité physique* : Si la demande est faible, +1GW ne change rien (on utilise du nucléaire pas cher). Mais si le réseau est saturé, +1GW oblige à démarrer une centrale à gaz ou charbon très chère, faisant exploser le prix. LightGBM capture ces seuils ("SI demande > 60GW ALORS prix +++").

2.  **Interactions Complexes** :
    LightGBM détecte automatiquement des combinaisons comme : *"S'il fait froid (demande chauffage) ET qu'il n'y a pas de vent (pas d'éolien), alors le prix s'envole"*. Un modèle classique doit être programmé manuellement pour voir cette interaction.

### Optimisation des Hyperparamètres (Grid Search)
Nous cherchons la meilleure configuration de l'algorithme pour minimiser l'erreur.

In [6]:
# Préparation Données Horaires
df_lgb = df.dropna().copy()

# Encodage des catégories (Saison)
for col in df_lgb.select_dtypes(include=['object']).columns:
    df_lgb[col] = df_lgb[col].astype('category').cat.codes

features = [c for c in df_lgb.columns if c not in [target_col, 'price_rolling_mean_24h', 'price_rolling_std_24h']]
X = df_lgb[features]
y = df_lgb[target_col]

# Échantillon récent pour l'optimisation 
X_sample = X.iloc[-20000:]
y_sample = y.iloc[-20000:]

param_grid = {
    'num_leaves': [31, 50, 70],         # Complexité des arbres
    'learning_rate': [0.01, 0.05, 0.1], # Vitesse d'apprentissage
    'n_estimators': [100, 200, 500],     # Nombre d'arbres
    'n_depth': [-1, 5, 10] # Profondeur 
}

gbm = lgb.LGBMRegressor(random_state=42, verbose=-1, n_jobs=-1)
tscv = TimeSeriesSplit(n_splits=3) # Validation croisée temporelle stricte

grid = GridSearchCV(gbm, param_grid, cv=tscv, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1)
grid.fit(X_sample, y_sample)

print(f"Meilleure config : {grid.best_params_}")


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Meilleure config : {'learning_rate': 0.05, 'n_depth': -1, 'n_estimators': 500, 'num_leaves': 50}


In [7]:

# Score MAE sur l'échantillon d'optimisation
best_preds = grid.predict(X_sample)
mae_lgb = mean_absolute_error(y_sample, best_preds)
rmse_lgb = root_mean_squared_error(y_sample, best_preds)
r2_lgb = r2_score(y_sample, best_preds)
mape_lgb = mean_absolute_percentage_error(y_sample, best_preds)
print(f"MAE LightGBM (Optimisé) : {mae_lgb:.2f} €/MWh | RMSE : {rmse_lgb:.2f} €/MWh | R2 : {r2_lgb:.2f} | MAPE : {mape_lgb:.2f}%")

MAE LightGBM (Optimisé) : 0.16 €/MWh | RMSE : 0.28 €/MWh | R2 : 1.00 | MAPE : 0.00%


## Analyse de la Volatilité : La courbe en "Crosse de Hockey"

Les prix de l'électricité sont connus pour leur volatilité extrême. Ce phénomène s'explique par la courbe d'offre du marché, souvent appelée **"Merit Order"**.

### Explication Économique
-   **La Base (Plat)** : Les premiers MW sont fournis par les EnR (Vent, Solaire) et le Nucléaire. Leur coût marginal est faible. Tant que la demande reste dans cette zone, le prix est bas et stable.
-   **La Pointe (Verticale)** : Quand la demande dépasse les capacités de base, on appelle les centrales à gaz ou charbon. Elles sont chères (coût du combustible + CO2). Dès qu'elles fixent le prix marginal, le prix du marché saute brutalement.

Le graphique ci-dessous visualise cette relation non-linéaire entre la **Charge (Load)** et le **Prix**.

In [8]:
df['volatility'] = df[target_col].rolling('24h').std()

# Définition des pics (> 95e percentile)
threshold = df[target_col].quantile(0.95)
df['status'] = df[target_col].apply(lambda x: 'Pic de Prix' if x > threshold else 'Normal')

fig = px.scatter(
    df.iloc[::10], # Échantillonnage pour lisibilité
    x='load',
    y=target_col,
    color='status',
    color_discrete_map={'Normal': 'lightgray', 'Pic de Prix': 'red'},
    title='Relation Charge vs Prix : La "Crosse de Hockey"',
    labels={'load': 'Consommation (MW)', 'price_day_ahead': 'Prix (€/MWh)'},
    opacity=0.6
)
fig.update_layout(template='plotly_white')
fig.show()

**Observation** : On voit clairement que pour une consommation faible (< 50GW), les points gris sont plats. Passé un certain seuil, les points rouges s'envolent verticalement. C'est la signature visuelle de la tension sur le réseau.

### Sauvegarde des modèles et des métadonnées

In [11]:
# Créer le dossier models s'il n'existe pas
models_dir = '../../models'
os.makedirs(models_dir, exist_ok=True)

# Sauvegarder le modèle SARIMAX
if 'results_sarima' in locals() or 'results_sarima' in globals():
    sarimax_path = os.path.join(models_dir, 'sarimax_france_2015_2017.pkl')
    joblib.dump(results_sarima, sarimax_path)
    print(f" Modèle SARIMAX sauvegardé: {sarimax_path}")
else:
    print(" Variable 'results_sarima' introuvable")

# Sauvegarder le modèle LightGBM optimisé (GridSearchCV)
if 'grid' in locals() or 'grid' in globals():
    lgbm_optimized_path = os.path.join(models_dir, 'lightgbm_france_2015_2017_gridsearch.pkl')
    joblib.dump(grid, lgbm_optimized_path)
    print(f" Modèle LightGBM GridSearch sauvegardé: {lgbm_optimized_path}")
    
    # Sauvegarder aussi le meilleur estimateur directement
    best_model_path = os.path.join(models_dir, 'lightgbm_france_2015_2017_best_estimator.pkl')
    joblib.dump(grid.best_estimator_, best_model_path)
    print(f" Meilleur estimateur LightGBM sauvegardé: {best_model_path}")
else:
    print(" Variable 'grid' introuvable")

# Sauvegarder les prédictions SARIMAX
if 'mean_forecast' in locals() or 'mean_forecast' in globals():
    forecast_path = os.path.join(models_dir, 'sarimax_france_2015_2017_forecast.pkl')
    forecast_data = {
        'predictions': mean_forecast,
        'confidence_interval': conf_int if 'conf_int' in locals() or 'conf_int' in globals() else None,
        'y_test': y_test if 'y_test' in locals() or 'y_test' in globals() else None
    }
    joblib.dump(forecast_data, forecast_path)
    print(f"Prédictions SARIMAX sauvegardées: {forecast_path}")

# Sauvegarder les métadonnées
metadata = {
    'model_name': 'SARIMAX + LightGBM Optimisé France 2015-2017',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset': 'df_features_france_2015_2017.csv',
    'target': 'price_day_ahead',
    
    # SARIMAX
    'sarimax': {
        'order': (1, 1, 1),
        'seasonal_order': (0, 1, 1, 7),
        'exog_vars': available_exog if 'available_exog' in locals() or 'available_exog' in globals() else [],
        'aggregation': 'Daily (mean)',
        'train_size': train_size if 'train_size' in locals() or 'train_size' in globals() else None,
        'train_days': len(train) if 'train' in locals() or 'train' in globals() else None,
        'test_days': len(test) if 'test' in locals() or 'test' in globals() else None,
        'metrics': {
            'MAE': float(mae) if 'mae' in locals() or 'mae' in globals() else None,
            'RMSE': float(rmse) if 'rmse' in locals() or 'rmse' in globals() else None,
            'R2': float(r2) if 'r2' in locals() or 'r2' in globals() else None,
            'MAPE': float(mape) if 'mape' in locals() or 'mape' in globals() else None
        }
    },
    
    # LightGBM
    'lightgbm': {
        'optimization_method': 'GridSearchCV',
        'cv_splits': 3,
        'cv_method': 'TimeSeriesSplit',
        'sample_size': len(X_sample) if 'X_sample' in locals() or 'X_sample' in globals() else None,
        'features': features if 'features' in locals() or 'features' in globals() else [],
        'n_features': len(features) if 'features' in locals() or 'features' in globals() else None,
        'best_params': grid.best_params_ if 'grid' in locals() or 'grid' in globals() else {},
        'param_grid': param_grid if 'param_grid' in locals() or 'param_grid' in globals() else {},
        'metrics': {
            'MAE': float(mae_lgb) if 'mae_lgb' in locals() or 'mae_lgb' in globals() else None,
            'RMSE': float(rmse_lgb) if 'rmse_lgb' in locals() or 'rmse_lgb' in globals() else None,
            'R2': float(r2_lgb) if 'r2_lgb' in locals() or 'r2_lgb' in globals() else None,
            'MAPE': float(mape_lgb) if 'mape_lgb' in locals() or 'mape_lgb' in globals() else None
        }
    },
    
    # Analyse de volatilité
    'volatility_analysis': {
        'threshold_percentile': 95,
        'threshold_value': float(threshold) if 'threshold' in locals() or 'threshold' in globals() else None
    }
}

metadata_path = os.path.join(models_dir, 'france_2015_2017_optimized_metadata.pkl')
joblib.dump(metadata, metadata_path)
print(f"Métadonnées sauvegardées: {metadata_path}")
print(f"📁 Tous les fichiers sauvegardés dans: {os.path.abspath(models_dir)}")


# Afficher un résumé
print("\nRÉSUMÉ DES MÉTRIQUES:")
print("\n🔹 SARIMAX (Journalier):")
if 'mae' in locals() or 'mae' in globals():
    print(f"   MAE:  {mae:.2f} €/MWh")
if 'rmse' in locals() or 'rmse' in globals():
    print(f"   RMSE: {rmse:.2f} €/MWh")
if 'r2' in locals() or 'r2' in globals():
    print(f"   R²:   {r2:.4f}")
if 'mape' in locals() or 'mape' in globals():
    print(f"   MAPE: {mape:.2f}%")

print("\n🔹 LightGBM Optimisé (Horaire):")
if 'mae_lgb' in locals() or 'mae_lgb' in globals():
    print(f"   MAE:  {mae_lgb:.2f} €/MWh")
if 'rmse_lgb' in locals() or 'rmse_lgb' in globals():
    print(f"   RMSE: {rmse_lgb:.2f} €/MWh")
if 'r2_lgb' in locals() or 'r2_lgb' in globals():
    print(f"   R²:   {r2_lgb:.4f}")
if 'mape_lgb' in locals() or 'mape_lgb' in globals():
    print(f"   MAPE: {mape_lgb:.2f}%")

if 'grid' in locals() or 'grid' in globals():
    print(f"\nMeilleurs hyperparamètres LightGBM:")
    for param, value in grid.best_params_.items():
        print(f"   {param}: {value}")

print("\nSauvegarde terminée avec succès!")

 Modèle SARIMAX sauvegardé: ../../models\sarimax_france_2015_2017.pkl
 Modèle LightGBM GridSearch sauvegardé: ../../models\lightgbm_france_2015_2017_gridsearch.pkl
 Meilleur estimateur LightGBM sauvegardé: ../../models\lightgbm_france_2015_2017_best_estimator.pkl
Prédictions SARIMAX sauvegardées: ../../models\sarimax_france_2015_2017_forecast.pkl
Métadonnées sauvegardées: ../../models\france_2015_2017_optimized_metadata.pkl
📁 Tous les fichiers sauvegardés dans: c:\Users\conta\Downloads\Projet_spe1\models

RÉSUMÉ DES MÉTRIQUES:

🔹 SARIMAX (Journalier):
   MAE:  4.76 €/MWh
   RMSE: 6.93 €/MWh
   R²:   0.6452
   MAPE: 0.07%

🔹 LightGBM Optimisé (Horaire):
   MAE:  0.16 €/MWh
   RMSE: 0.28 €/MWh
   R²:   0.9997
   MAPE: 0.00%

Meilleurs hyperparamètres LightGBM:
   learning_rate: 0.05
   n_depth: -1
   n_estimators: 500
   num_leaves: 50

Sauvegarde terminée avec succès!
